In [7]:
%load_ext autoreload
%autoreload 2
from building_dection import *

import os
import rasterio as rio
import numpy as np
import json
import pyproj
from osgeo import gdal
import subprocess
import zipfile

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, LTAError, LTATriggered
from datetime import date

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
sn7_path = "/scratch/toml/sn7"
train_path = os.path.join(sn7_path, "SN7_buildings_train/train")
sentinel_path = os.path.join(sn7_path, "sentinel2")

In [9]:
labels = get_labels(train_path)[:20]

In [18]:
products = get_sentinel2_product_ids(labels[:5])

L15-0331E-1257N_1327_3160_13: POLYGON((-121.68466515096087 37.9615752659464, -121.64058492652846 37.9615752659464, -121.64058492652846 37.99617689154867, -121.68466515096087 37.99617689154867, -121.68466515096087 37.9615752659464))
Error: HTTP status 200 OK: 
The Sentinels Scientific Data Hub

# Copernicus Sentinel Data is now available on the Copernicus Data Space
Ecosystem

[https://dataspace.copernicus.eu](https://dataspace.copernicus.eu/)
L15-0357E-1223N_1429_3296_13: POLYGON((-117.20231626982802 33.100719225768806, -117.15811895229255 33.100719225768806, -117.15811895229255 33.13760723935068, -117.20231626982802 33.13760723935068, -117.20231626982802 33.100719225768806))
Error: HTTP status 200 OK: 
The Sentinels Scientific Data Hub

# Copernicus Sentinel Data is now available on the Copernicus Data Space
Ecosystem

[https://dataspace.copernicus.eu](https://dataspace.copernicus.eu/)
L15-0358E-1220N_1433_3310_13: POLYGON((-117.02648733393058 32.58380527888377, -116.9824193392965 32.

In [20]:
downloaded_prods = get_sentinel2_products(products, sentinel_path)

In [12]:
downloaded_prods = [dp for dp in downloaded_prods if dp is not None]

now = datetime.now().strftime("%Y%m%d_%H%M%S")
info_json_fn = f"{sentinel_path}/product_infos_{now}.json"
with open(os.path.join(sentinel_path, "product_infos.json"), "w") as f:
    json.dump(downloaded_prods, f)

In [13]:
locs = {}
for prod in downloaded_prods:
    locs[prod['label']] = f"{prod['title']}.SAFE"

modifications = [
    {
        "flag": "SEN2COR_MODIFY_LUT_AEROSOL",
        "value": "MARITIME",
        "info": "Look Up Table Aerosol set to MARITIME",
        "name": "LUT_AEROSOL_MARITIME",
    },
    # {
    #     "flag": "SEN2COR_MODIFY_LUT_SEASON",
    #     "value": "WINTER",
    #     "info": "Look Up Table Mid Latitude Season set to WINTER",
    #     "name": "LUT_SEASON_WINTER",
    # },
]


In [14]:
%autoreload 2
from l2a_analysis import L2A_Analysis

l2a_analysis = L2A_Analysis(report_name="space_net_7_10_2020", base_input_dir="/scratch/toml/sn7/sentinel2", base_output_dir="/scratch/toml/sn7/reports", resolution=10)
l2a_analysis.set_locations(locs)
l2a_analysis.set_modifications(modifications)


In [15]:
l2a_analysis.run_sen2cor()

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/toml/sn7/reports/space_net_7_10_2020/data_info.json'

In [ ]:
l2a_analysis.read_l2a_data()

Added band B01 to stack
Added band B02 to stack
Added band B03 to stack
Added band B04 to stack
Added band B05 to stack
Added band B06 to stack
Added band B07 to stack
Added band B8A to stack
Added band B09 to stack
Added band B11 to stack
Added band B12 to stack
Added band SCL to stack
Added band AOT to stack
Added band WVP to stack
Added band TCI to stack
Added band B01 to stack
Added band B02 to stack
Added band B03 to stack
Added band B04 to stack
Added band B05 to stack
Added band B06 to stack
Added band B07 to stack
Added band B8A to stack
Added band B09 to stack
Added band B11 to stack
Added band B12 to stack
Added band SCL to stack
Added band AOT to stack
Added band WVP to stack
Added band TCI to stack
Added band B01 to stack
Added band B02 to stack
Added band B03 to stack
Added band B04 to stack
Added band B05 to stack
Added band B06 to stack
Added band B07 to stack
Added band B8A to stack
Added band B09 to stack
Added band B11 to stack
Added band B12 to stack
Added band SCL t

In [ ]:
# rasterize labels, crop to extent of sentinel products, save as tif and orgainize in train folder

train_base_dir = "/scratch/toml/sn7/train"

for prod in downloaded_prods:
    label = prod["label"]
    label_dir = os.path.join(train_base_dir, label)
    if os.path.exists(label_dir) == False:
        os.mkdir(label_dir)
        os.mkdir(os.path.join(label_dir, "images"))
        
        # reproject and rasterize building labels
        reference_band = l2a_analysis.reference_bands[label]['B02']
        with rio.open(reference_band.jp2_path) as src:
            extent = src.bounds
            tgt_crs = src.crs.to_string()

        # all building labels are in epsg:4326
        src_crs = 'EPSG:4326'
        
        label_geojsons = os.listdir(f"{train_path}/{label}/labels")
        building_labels_geojson = label_geojsons[0]
        original_labels = f"{train_path}/{label}/labels/{building_labels_geojson}"
        extent_file = f"{train_path}/{label}/images/{os.listdir(f'{train_path}/{label}/images/')[0]}" 
        reprojected_labels = f"{label_dir}/{building_labels_geojson}"
        rasterized_labels = f"{reprojected_labels.split('.')[0]}.tif"
        cropped_labels = f"{rasterized_labels.split('.')[0]}_cropped.tif"
        
        from building_dection import reproject_geojson, rasterize_labels, crop_to_extent
        reproject_geojson(original_labels, src_crs, tgt_crs, out_fn=reprojected_labels)
        rasterize_labels(reprojected_labels, rasterized_labels, extent)
        crop_to_extent(original_labels, rasterized_labels, cropped_labels)
        # rasterize labels

        ref_image_dir = os.path.join(label_dir, "images","ref") 
        os.mkdir(ref_image_dir)
        for band_name in l2a_analysis.reference_bands[label].bands:
            band = l2a_analysis.reference_bands[label][band_name]
            src_path = band.jp2_path
            tgt_path = f"{ref_image_dir}/{band.name}.tif"
            crop_to_extent(original_labels,src_path, tgt_path)
        for mod in l2a_analysis.modifications:
            mod_image_dir = os.path.join(label_dir, "images", mod["name"])
            os.mkdir(mod_image_dir)
            for band_name in l2a_analysis.modified_bands[label][mod["name"]].bands:
                band = l2a_analysis.modified_bands[label][mod["name"]][band_name]
                src_path = band.jp2_path
                tgt_path = f"{mod_image_dir}/{band.name}.tif"
                crop_to_extent(original_labels,src_path, tgt_path)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
def get_extent_raster(raster_fn):
    with rio.open(raster_fn) as ds:
        extent = ds.bounds
        extent_crs = ds.crs
    return extent, extent_crs

def get_extent_vector(vector_fn):
    vector_ds = gdal.OpenEx(vector_fn, gdal.OF_VECTOR)
    vector_extent = vector_ds.GetLayer().GetExtent()
    vector_crs = pyproj.CRS.from_wkt(vector_ds.GetLayer().GetSpatialRef().__str__())
    return vector_extent, vector_crs

geojson_1 = "/scratch/toml/sn7/SN7_buildings_train/train/L15-0331E-1257N_1327_3160_13/labels/global_monthly_2018_01_mosaic_L15-0331E-1257N_1327_3160_13_Buildings.geojson"
geojson_2 = "/scratch/toml/sn7/SN7_buildings_train/train/L15-0614E-0946N_2459_4406_13/labels/global_monthly_2018_03_mosaic_L15-0614E-0946N_2459_4406_13_Buildings.geojson"
geojson_3 = "/scratch/toml/sn7/SN7_buildings_train/train/L15-0566E-1185N_2265_3451_13/labels/global_monthly_2018_01_mosaic_L15-0566E-1185N_2265_3451_13_Buildings.geojson"

geojsons = [geojson_1, geojson_2, geojson_3]
for g in geojsons:
    extent, crs = get_extent_vector(g)
    # print(extent)
    print(f"x: {extent[1] - extent[0]}, y: {extent[3] - extent[2]}")

raster_1 = "/scratch/toml/sn7/SN7_buildings_train/train/L15-0331E-1257N_1327_3160_13/images/global_monthly_2018_01_mosaic_L15-0331E-1257N_1327_3160_13.tif"
raster_2 = "/scratch/toml/sn7/SN7_buildings_train/train/L15-0357E-1223N_1429_3296_13/images/global_monthly_2018_01_mosaic_L15-0357E-1223N_1429_3296_13.tif"
raster_3 = "/scratch/toml/sn7/SN7_buildings_train/train/L15-0358E-1220N_1433_3310_13/images/global_monthly_2018_02_mosaic_L15-0358E-1220N_1433_3310_13.tif"

rasters = [raster_1, raster_2, raster_3]

for r in rasters:
    extent, crs = get_extent_raster(r)
    print(extent)
    print(f"x: {extent[2] - extent[0]}, y: {extent[3] - extent[1]}")


x: 0.04408022443240611, y: 0.03460162560227076
x: 0.04402370195680305, y: 0.041665185990940046
x: 0.01742558826303764, y: 0.007187311416796405
BoundingBox(left=-13545864.404585794, bottom=4573996.549899891, right=-13540972.434776222, top=4578883.742395196)
x: 4891.969809571281, y: 4887.192495305091
BoundingBox(left=-13046883.483940167, bottom=3908688.655705719, right=-13041991.514130596, top=3913575.8482010234)
x: 4891.969809571281, y: 4887.1924953046255
BoundingBox(left=-13027315.604699157, bottom=3840196.3010479314, right=-13022423.634889586, top=3845088.270857503)
x: 4891.969809571281, y: 4891.969809571747
